## Movie Revenue Prediction 

### Objective: Your client is a movie studio and they need to be able to predict movie revenue in order to greenlight the project and assign a budget to it. 
- Most of the data is comprised of categorical variables. 
- While the budget for the movie is known in the dataset, it is often an unknown variable during the greenlighting process. 

In [1]:
%load_ext watermark
%watermark -a "Emily Schoof" -d -t -v -p numpy,pandas,matplotlib

Emily Schoof 2019-08-19 10:01:09 

CPython 3.7.3
IPython 7.4.0

numpy 1.16.2
pandas 0.24.2
matplotlib 3.0.3


## Section 1: Data Preprocessing
- Load Movie_Revenue_Predictions.csv data
- Cleaning Data and Exploration

In [2]:
# Import necessary modules
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
# Load the dataset
movie_df = DataFrame(pd.read_csv('Movie_Revenue_Predictions.csv'))
movie_df.head()

,title,tagline,revenue,budget,genres,homepage,id,keywords,original_language,overview,production_companies,production_countries,release_date,runtime,spoken_languages,status
0,Avatar,Enter the World of Pandora.,2787965087,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",12/10/09,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released
1,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.",961000000,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",5/19/07,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released
2,Spectre,A Plan No One Escapes,880674609,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",10/26/15,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released
3,The Dark Knight Rises,The Legend Ends,1084939099,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",7/16/12,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released
4,John Carter,"Lost in our world, found in another.",284139100,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",3/7/12,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released


In [4]:
# Assess shape of data
movie_df.shape

(4803, 16)

In [5]:
# Assess dataframe
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 16 columns):
title                   4803 non-null object
tagline                 3959 non-null object
revenue                 4803 non-null int64
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
overview                4800 non-null object
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
dtypes: float64(1), int64(3), object(12)
memory usage: 600.5+ KB


In [6]:
movie_df.isnull().sum()

title                      0
tagline                  844
revenue                    0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
overview                   3
production_companies       0
production_countries       0
release_date               1
runtime                    2
spoken_languages           0
status                     0
dtype: int64

## Plan of Action
#### 1. Numerical columns:
   - **runtime** (2 NaN)
       - since there are so few instances, dropping the NaN rows will not interfere with the analysis of movie revenue. 
   - **budget**
       - since budget often an unknown variable during the greenlighting process, it should not be a feature in a model to predict revenue. 
   - **id**
       - this column does not appear to have any tangible data, so it can be dropped. 
        
#### 2. Categorical/Object columns:
Filling in NaN categorical values in the remaining columns is a bit tricky since there is no easily-applied statistical method.
   - **title** 
       - Since the movie title makes an initial impact on a consumer's decision to see a movie, NLP processing will be applied to this column in order to assess if certain words and the overall length of the title has an impact on the resulting movie revenue.
   - **homepage** (3091 NaN)
        - 3/4 of the data is missing (3091 of the total 4803), so this column cannot be effectively utilized for this model and should be dropped.
   - **overview** (3 NaN)
        - since there are so few instances, dropping the NaN rows will not interfere with the analysis of movie revenue. However, since there is already a column with keywords, this should be dropped. Like the title column, NLP can be applied to this column as well. 
   - **release_date** (1 NaN)
        - since there are so few instances, dropping the NaN rows will not interfere with the analysis of movie revenue. 
   - **tagline** (844 NaN)
        - while 844 is well under 1/4 of the total data, the tagline for a movie may in fact have a significant impact on movie revenue due to its marketing implications. However, since there is already a column with keywords, this should be dropped.*

In [7]:
# Drop NaNs in Numerical Columns - only select rows where runtime and release_date columns are "not null"
movie_df = movie_df.dropna(subset=['runtime', 'release_date'])
len(movie_df)

4800

In [8]:
# Convert Dates to Datetime Objects
movie_df['release_date_dt'] = pd.to_datetime(movie_df['release_date'], infer_datetime_format=True)
movie_df['release_date_dt'].head(2)

0   2009-12-10
1   2007-05-19
Name: release_date_dt, dtype: datetime64[ns]

In [9]:
# Drop Unnecessary Columns and Columns with Too Many NaNs *(> 50% of entries)* to Resolve
movie_df = movie_df.drop(columns=['id', 'budget', 'tagline', 'homepage', 'release_date'])
movie_df.head(2)

,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
0,Avatar,2787965087,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2009-12-10
1,Pirates of the Caribbean: At World's End,961000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2007-05-19


In [10]:
# Test output
movie_df.isnull().sum()

title                   0
revenue                 0
genres                  0
keywords                0
original_language       0
overview                1
production_companies    0
production_countries    0
runtime                 0
spoken_languages        0
status                  0
release_date_dt         0
dtype: int64

## Explore the Values within Each Column

In [11]:
# Import helpful modules
from collections import Counter

**Title**

In [12]:
print(len(movie_df.title.value_counts(ascending=False)))
print(movie_df.title.describe())
movie_df.title.value_counts(ascending=False).nlargest()

4797
count         4800
unique        4797
top       The Host
freq             2
Name: title, dtype: object


The Host                      2
Batman                        2
Out of the Blue               2
Survivor                      1
Things We Lost in the Fire    1
Name: title, dtype: int64

**Revenue**

In [13]:
print(len(movie_df.revenue.value_counts(ascending=False)))
print(movie_df.revenue.describe())
movie_df.revenue.value_counts(ascending=False).nlargest()

3297
count    4.800000e+03
mean     8.231205e+07
std      1.628950e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.918199e+07
75%      9.293886e+07
max      2.787965e+09
Name: revenue, dtype: float64


0           1424
7000000        6
8000000        6
6000000        5
12000000       5
Name: revenue, dtype: int64

*There are 1424 movies within the dataset that have a reported $0 for revenue. This is most likely due to missing data. Since movie revenue is the target variable, I'm going to convert all the '0' entries to NaNs, then use the **Imputer** module to replace NaNs with the mean revenue.*

In [14]:
movie_df['revenue'] = movie_df['revenue']. replace(0, np.nan)

**Genres**

In [15]:
print(movie_df.genres.describe())
print(len(movie_df.genres.value_counts(ascending=False)))
most_common, num_most_common = Counter(movie_df.genres).most_common(1)[0]
print(most_common, num_most_common)
movie_df.genres.value_counts(ascending=False).nlargest()

count                              4800
unique                             1175
top       [{"id": 18, "name": "Drama"}]
freq                                369
Name: genres, dtype: object
1175
[{"id": 18, "name": "Drama"}] 369


[{"id": 18, "name": "Drama"}]                                       369
[{"id": 35, "name": "Comedy"}]                                      282
[{"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}]     164
[{"id": 35, "name": "Comedy"}, {"id": 10749, "name": "Romance"}]    144
[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}]         142
Name: genres, dtype: int64

**Keywords**

In [16]:
print(len(movie_df.keywords.value_counts(ascending=False)))
print(movie_df.keywords.describe())
most_common, num_most_common = Counter(movie_df.keywords).most_common(1)[0]
print(most_common, num_most_common)
movie_df.keywords.value_counts(ascending=False).nlargest()

4220
count     4800
unique    4220
top         []
freq       411
Name: keywords, dtype: object
[] 411


[]                                                  411
[{"id": 10183, "name": "independent film"}]          55
[{"id": 187056, "name": "woman director"}]           42
[{"id": 179431, "name": "duringcreditsstinger"}]     15
[{"id": 6075, "name": "sport"}]                      13
Name: keywords, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the most common keyword is '[ ]', which is empty and holds no value.*

**Original Language**

In [17]:
print(len(movie_df.original_language.value_counts(ascending=False)))
print(movie_df.original_language.describe())
movie_df.original_language.value_counts(ascending=False).nlargest()

37
count     4800
unique      37
top         en
freq      4503
Name: original_language, dtype: object


en    4503
fr      70
es      32
zh      27
de      27
Name: original_language, dtype: int64

**Production Companies**

In [18]:
print(len(movie_df.production_companies.value_counts(ascending=False)))
print(movie_df.production_companies.describe())
most_common, num_most_common = Counter(movie_df.production_companies).most_common(1)[0]
print(most_common, num_most_common)
movie_df.production_companies.value_counts(ascending=False).nlargest()

3695
count     4800
unique    3695
top         []
freq       350
Name: production_companies, dtype: object
[] 350


[]                                            350
[{"name": "Paramount Pictures", "id": 4}]      58
[{"name": "Universal Pictures", "id": 33}]     45
[{"name": "New Line Cinema", "id": 12}]        38
[{"name": "Columbia Pictures", "id": 5}]       37
Name: production_companies, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the most common production_companies is '[ ]', which is empty and holds no value.*

In [19]:
no_company = movie_df[movie_df['production_companies'] == '[]']
print(len(no_company))
no_company.head()

350


,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
1011,The Tooth Fairy,NaN,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 10292, ""name"": ""gore""}, {""id"": 12339, ...",de,A woman and her daughter (Nicole Muñoz) encoun...,[],[],0.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2006-08-08
1360,There Be Dragons,NaN,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 5509, ""name"": ""spanish civil war""}, {""...",en,Arising out of the horror of the Spanish Civil...,[],[],112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2011-03-25
1669,The Promise,NaN,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 18, ""na...","[{""id"": 964, ""name"": ""servant""}, {""id"": 2280, ...",zh,"An orphaned girl, driven by poverty at such a ...",[],"[{""iso_3166_1"": ""CN"", ""name"": ""China""}, {""iso_...",98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2005-12-15
1754,The Sisterhood of the Traveling Pants 2,44352417.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 35, ""...","[{""id"": 5248, ""name"": ""female friendship""}, {""...",en,Four young women continue the journey toward a...,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",117.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2008-08-06
1898,Unaccompanied Minors,NaN,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 65, ""name"": ""holiday""}]",en,Five disparate kids snowed in at the airport o...,[],[],90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2006-12-08


**Production Countries**

In [20]:
print(len(movie_df.production_countries.value_counts(ascending=False)))
print(movie_df.production_countries.describe())
most_common, num_most_common = Counter(movie_df.production_countries).most_common(1)[0]
print(most_common, num_most_common)
movie_df.production_countries.value_counts(ascending=False).nlargest()

469
count                                                  4800
unique                                                  469
top       [{"iso_3166_1": "US", "name": "United States o...
freq                                                   2977
Name: production_countries, dtype: object
[{"iso_3166_1": "US", "name": "United States of America"}] 2977


[{"iso_3166_1": "US", "name": "United States of America"}]                                                    2977
[{"iso_3166_1": "GB", "name": "United Kingdom"}, {"iso_3166_1": "US", "name": "United States of America"}]     181
[]                                                                                                             173
[{"iso_3166_1": "GB", "name": "United Kingdom"}]                                                               130
[{"iso_3166_1": "DE", "name": "Germany"}, {"iso_3166_1": "US", "name": "United States of America"}]            119
Name: production_countries, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the incidence of 'US' (on it's own) as the production country appears to be the most common, making up around 50% of the dataset (2977 out of 4799). In addition, there are 172 entries of '[]' that appear to hold no information.*

**Runtime**

In [21]:
print(len(movie_df.runtime.value_counts(ascending=False)))
print(movie_df.runtime.describe())
movie_df.runtime.value_counts(ascending=False).nlargest()

156
count    4800.000000
mean      106.898125
std        22.561593
min         0.000000
25%        94.000000
50%       103.000000
75%       118.000000
max       338.000000
Name: runtime, dtype: float64


90.0     163
100.0    149
98.0     140
97.0     133
95.0     123
Name: runtime, dtype: int64

*There appear to be movies with a '0.0' runtime, which doesn't make sense, since this would imply that the movie was 0 minutes long.*

In [22]:
no_runtime = movie_df[movie_df['runtime'] == 0.0]
print(len(no_runtime))
no_runtime.head()

34


,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
1011,The Tooth Fairy,NaN,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 10292, ""name"": ""gore""}, {""id"": 12339, ...",de,A woman and her daughter (Nicole Muñoz) encoun...,[],[],0.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2006-08-08
3112,Blood Done Sign My Name,NaN,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",[],en,A drama based on the true story in which a bla...,[],[],0.0,[],Released,2010-02-01
3669,Should've Been Romeo,NaN,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",[],en,"A self-centered, middle-aged pitchman for a po...","[{""name"": ""Phillybrook Films"", ""id"": 65147}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2012-04-28
3809,How to Fall in Love,NaN,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],en,"An accountant, who never quite grew out of his...","[{""name"": ""Annuit Coeptis Entertainment Inc."",...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",0.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2012-07-21
3953,Fort McCoy,NaN,"[{""id"": 10752, ""name"": ""War""}, {""id"": 18, ""nam...","[{""id"": 187056, ""name"": ""woman director""}]",en,Unable to serve in World War II because of a h...,[],[],0.0,[],Released,2014-01-01


**Spoken Languages**

In [23]:
print(len(movie_df.spoken_languages.value_counts(ascending=False)))
print(movie_df.spoken_languages.describe())
most_common, num_most_common = Counter(movie_df.spoken_languages).most_common(1)[0]
print(most_common, num_most_common)
movie_df.spoken_languages.value_counts(ascending=False).nlargest()

544
count                                         4800
unique                                         544
top       [{"iso_639_1": "en", "name": "English"}]
freq                                          3171
Name: spoken_languages, dtype: object
[{"iso_639_1": "en", "name": "English"}] 3171


[{"iso_639_1": "en", "name": "English"}]                                                  3171
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\u00f1ol"}]      127
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "fr", "name": "Fran\u00e7ais"}]     114
[]                                                                                          84
[{"iso_639_1": "es", "name": "Espa\u00f1ol"}, {"iso_639_1": "en", "name": "English"}]       54
Name: spoken_languages, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the incidence of 'English' as the spoken language appears to be the most common, making up around 75% of the dataset (3170 out of 4799). There are also 84 entries with a '[]'.*

In [24]:
no_language = movie_df[movie_df['spoken_languages'] == '[]']
print(len(no_language))
no_language.head()

84


,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
492,Top Cat Begins,NaN,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 16, ""nam...","[{""id"": 209714, ""name"": ""3d""}]",es,Top Cat has arrived to charm his way into your...,"[{""name"": ""Anima Estudios"", ""id"": 9965}, {""nam...","[{""iso_3166_1"": ""IN"", ""name"": ""India""}, {""iso_...",89.0,[],Released,2015-10-30
1169,42,95020213.0,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1480, ""name"": ""baseball""}, {""id"": 5565...",en,"The powerful story of Jackie Robinson, the leg...","[{""name"": ""Warner Bros."", ""id"": 6194}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",128.0,[],Released,2013-04-12
2590,VeggieTales: The Pirates Who Don't Do Anything,NaN,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""...","[{""id"": 380, ""name"": ""brother brother relation...",en,Set Sail For Adventure! A boatload of beloved ...,"[{""name"": ""Starz Animation"", ""id"": 2885}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",85.0,[],Released,2008-01-11
2614,The Love Letter,NaN,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",[],en,20th century computer games designer Scott exc...,[],[],98.0,[],Released,1998-02-01
2631,The Company,NaN,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 11162, ""name"": ""miniseries""}]",en,Real-life figures from the Cold War era mix wi...,[],[],276.0,[],Released,2007-08-05


**Status**

In [25]:
print(len(movie_df.status.value_counts(ascending=False)))
print(movie_df.status.describe())
print(movie_df.status.value_counts(ascending=False))
most_common, num_most_common = Counter(movie_df.status).most_common(1)[0]
most_common, num_most_common

3
count         4800
unique           3
top       Released
freq          4792
Name: status, dtype: object
Released           4792
Rumored               5
Post Production       3
Name: status, dtype: int64


('Released', 4792)

**Release Date**

In [26]:
print(len(movie_df.release_date_dt.value_counts(ascending=False)))
print(movie_df.release_date_dt.describe()) 
most_common, num_most_common = Counter(movie_df.release_date_dt).most_common(1)[0]
least_common, num_least_common = Counter(movie_df.release_date_dt).most_common()[-1]
print(most_common, num_most_common, least_common, num_least_common)
movie_df.release_date_dt.value_counts(ascending=False).nlargest()

3279
count                    4800
unique                   3279
top       2006-01-01 00:00:00
freq                       10
first     1969-01-01 00:00:00
last      2068-12-21 00:00:00
Name: release_date_dt, dtype: object
2006-01-01 00:00:00 10 2012-05-03 00:00:00 1


2006-01-01    10
2002-01-01     8
2013-07-18     7
2004-09-03     7
1999-10-22     7
Name: release_date_dt, dtype: int64

*The column dates range from the years 1969 to the future year of 2068. This is interesting, since movie revenue cannot be recorded for future dates, but it can be predicted (as is the goal of this analysis). Before moving forward, the movies with dates greater than 2020 should be dropped.*

In [27]:
movie_df = movie_df.loc[movie_df.release_date_dt < '2020-01-01 00:00:00']
print(len(movie_df.release_date_dt.value_counts(ascending=False)))
print(movie_df.release_date_dt.describe())

3148
count                    4668
unique                   3148
top       2006-01-01 00:00:00
freq                       10
first     1969-01-01 00:00:00
last      2017-02-03 00:00:00
Name: release_date_dt, dtype: object


This looks much better. It appears data collection for this dataset must have stopped in early 2017, since no movies released in 2018 and 2019 are present in the dataset despite the maximum year elligibility being 2019.

### Drop row with the following 6 conditions:
    1. movie.revenue.isnull() == True
    2. movie.keyword == []
    3. movie.production_companies == []
    4. movie_df.runtime == 0.0
If all 6 of these conditions are met, then the resulting impact on a model built to predict a movie's revenue will be weakened; thus, these rows should be removed entirely.

In [28]:
# Drop rows with sub-optimal conditions:

# revenue of NaN and runtime of 0
movie_df = movie_df.drop(
     movie_df[(movie_df.revenue.isnull() == True) & 
              (movie_df.runtime == 0.0)].index)

# revenue of NaN and empty keyword, production company, and null tagline
movie_df = movie_df.drop(
     movie_df[(movie_df.revenue.isnull() == True) & 
              (movie_df.keywords == "[]") & 
              (movie_df.production_companies == "[]")].index)

# Drop rows where 
len(movie_df)

4523

In [29]:
# Sort Values by revenue
movie_df = movie_df.sort_values(by='revenue', ascending=False)

#Reset Index
movie_df = movie_df.reset_index()
movie_df.head(2)

,index,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
0,0,Avatar,2.787965e+09,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2009-12-10
1,25,Titanic,1.845034e+09,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 2580, ""name"": ""shipwreck""}, {""id"": 298...",en,"84 years later, a 101-year-old woman named Ros...","[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",194.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,1997-11-18


In [30]:
# Drop index
movie_df = movie_df.drop(columns='index')
movie_df.head(2)

,title,revenue,genres,keywords,original_language,overview,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
0,Avatar,2.787965e+09,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2009-12-10
1,Titanic,1.845034e+09,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 2580, ""name"": ""shipwreck""}, {""id"": 298...",en,"84 years later, a 101-year-old woman named Ros...","[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",194.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,1997-11-18


In [33]:
# Store dataframe globally
%store movie_df

Stored 'movie_df' (DataFrame)
